# 自然言語処理
非構造データだが統計的な処理をされるものとして自然言語処理がある。

本項では「形態素解析」「品詞解析」「分かち書き」「構文解析」など基本的な事に加えてTF-IDFとそれを使用したトピック分析を行う

## ライブラリのインポート

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.datasets import fetch_20newsgroups
from janome.tokenizer import Tokenizer
from googletrans import Translator
import collections
import nltk

## 形態素解析
日本語での形態素解析を行う。ここで使用する文章はWikipediaの『自然言語処理』とする。

In [2]:
f = open("sample.txt", encoding="utf-8")
text = f.read()
text

'自然言語処理（しぜんげんごしょり、英語: natural language processing、略称：NLP）は、人間が日常的に使っている自然言語をコンピュータに処理させる一連の技術であり、人工知能と言語学の一分野である。「計算言語学」（computational linguistics）との類似もあるが、自然言語処理は工学的な視点からの言語処理をさすのに対して、計算言語学は言語学的視点を重視する手法をさす事が多い[1]。データベース内の情報を自然言語に変換したり、自然言語の文章をより形式的な（コンピュータが理解しやすい）表現に変換するといった処理が含まれる。応用例としては予測変換、IMEなどの文字変換が挙げられる。\n自然言語の理解をコンピュータにさせることは、自然言語理解とされている。自然言語理解と、自然言語処理の差は、意味を扱うか、扱わないかという説もあったが、最近は数理的な言語解析手法（統計や確率など）が広められた為、パーサ（統語解析器）などが一段と精度や速度が上がり、その意味合いは違ってきている。もともと自然言語の意味論的側面を全く無視して達成できることは非常に限られている。このため、自然言語処理には形態素解析と構文解析、文脈解析、意味解析などをSyntaxなど表層的な観点から解析をする学問であるが、自然言語理解は、意味をどのように理解するかという個々人の理解と推論部分が主な研究の課題になってきており、両者の境界は意思や意図が含まれるかどうかになってきている。 '

### 形態素解析

In [3]:
t = Tokenizer()
for token in t.tokenize(text):
    print(token)

自然	名詞,形容動詞語幹,*,*,*,*,自然,シゼン,シゼン
言語	名詞,一般,*,*,*,*,言語,ゲンゴ,ゲンゴ
処理	名詞,サ変接続,*,*,*,*,処理,ショリ,ショリ
（	記号,括弧開,*,*,*,*,（,（,（
し	動詞,自立,*,*,サ変・スル,連用形,する,シ,シ
ぜん	名詞,一般,*,*,*,*,ぜん,ゼン,ゼン
げん	名詞,一般,*,*,*,*,げん,ゲン,ゲン
ごしょ	動詞,自立,*,*,サ変・−スル,未然ウ接続,ごする,ゴショ,ゴショ
り	助動詞,*,*,*,文語・リ,基本形,り,リ,リ
、	記号,読点,*,*,*,*,、,、,、
英語	名詞,一般,*,*,*,*,英語,エイゴ,エイゴ
:	名詞,サ変接続,*,*,*,*,:,*,*
 	記号,空白,*,*,*,*, ,*,*
natural	名詞,固有名詞,組織,*,*,*,natural,*,*
 	記号,空白,*,*,*,*, ,*,*
language	名詞,固有名詞,組織,*,*,*,language,*,*
 	記号,空白,*,*,*,*, ,*,*
processing	名詞,固有名詞,組織,*,*,*,processing,*,*
、	記号,読点,*,*,*,*,、,、,、
略称	名詞,サ変接続,*,*,*,*,略称,リャクショウ,リャクショー
：	記号,一般,*,*,*,*,：,：,：
NLP	名詞,固有名詞,組織,*,*,*,NLP,*,*
）	記号,括弧閉,*,*,*,*,）,）,）
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
、	記号,読点,*,*,*,*,、,、,、
人間	名詞,一般,*,*,*,*,人間,ニンゲン,ニンゲン
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
日常	名詞,一般,*,*,*,*,日常,ニチジョウ,ニチジョー
的	名詞,接尾,形容動詞語幹,*,*,*,的,テキ,テキ
に	助詞,副詞化,*,*,*,*,に,ニ,ニ
使っ	動詞,自立,*,*,五段・ワ行促音便,連用タ接続,使う,ツカッ,ツカッ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
いる	動詞,非自立,*,*,一段,基本形,いる,イル,イル
自然	名詞,形容動詞語幹,*,*,*,*,自然,シゼン,シゼン
言語	名詞,一般,*,*,*,*,言語,ゲンゴ,ゲンゴ
を	助詞,格助詞,一般,*

### 分かち書き
機械学習などで日本語を特徴量とする際に単語で分解してリストに格納することで後述するTF-IDFに使用する。

In [4]:
wakati = list(t.tokenize(text, wakati=True))
wakati

['自然',
 '言語',
 '処理',
 '（',
 'し',
 'ぜん',
 'げん',
 'ごしょ',
 'り',
 '、',
 '英語',
 ':',
 ' ',
 'natural',
 ' ',
 'language',
 ' ',
 'processing',
 '、',
 '略称',
 '：',
 'NLP',
 '）',
 'は',
 '、',
 '人間',
 'が',
 '日常',
 '的',
 'に',
 '使っ',
 'て',
 'いる',
 '自然',
 '言語',
 'を',
 'コンピュータ',
 'に',
 '処理',
 'さ',
 'せる',
 '一連',
 'の',
 '技術',
 'で',
 'あり',
 '、',
 '人工',
 '知能',
 'と',
 '言語',
 '学',
 'の',
 '一',
 '分野',
 'で',
 'ある',
 '。',
 '「',
 '計算',
 '言語',
 '学',
 '」',
 '（',
 'computational',
 ' ',
 'linguistics',
 '）',
 'と',
 'の',
 '類似',
 'も',
 'ある',
 'が',
 '、',
 '自然',
 '言語',
 '処理',
 'は',
 '工学',
 '的',
 'な',
 '視点',
 'から',
 'の',
 '言語',
 '処理',
 'を',
 'さす',
 'の',
 'に対して',
 '、',
 '計算',
 '言語',
 '学',
 'は',
 '言語',
 '学',
 '的',
 '視点',
 'を',
 '重視',
 'する',
 '手法',
 'を',
 'さす',
 '事',
 'が',
 '多い',
 '[',
 '1',
 ']。',
 'データベース',
 '内',
 'の',
 '情報',
 'を',
 '自然',
 '言語',
 'に',
 '変換',
 'し',
 'たり',
 '、',
 '自然',
 '言語',
 'の',
 '文章',
 'を',
 'より',
 '形式',
 '的',
 'な',
 '（',
 'コンピュータ',
 'が',
 '理解',
 'し',
 'やすい',
 '）',
 '表現',
 'に',
 '変換',
 'する',
 'といった',

### 品詞
分かち書きされて配列に格納された単語に対して品詞を解析する。

In [5]:
speech = []
for token in t.tokenize(text):
    speech.append(token.part_of_speech.split(',')[0])
print(speech)

['名詞', '名詞', '名詞', '記号', '動詞', '名詞', '名詞', '動詞', '助動詞', '記号', '名詞', '名詞', '記号', '名詞', '記号', '名詞', '記号', '名詞', '記号', '名詞', '記号', '名詞', '記号', '助詞', '記号', '名詞', '助詞', '名詞', '名詞', '助詞', '動詞', '助詞', '動詞', '名詞', '名詞', '助詞', '名詞', '助詞', '名詞', '動詞', '動詞', '名詞', '助詞', '名詞', '助動詞', '助動詞', '記号', '名詞', '名詞', '助詞', '名詞', '名詞', '助詞', '名詞', '名詞', '助動詞', '助動詞', '記号', '記号', '名詞', '名詞', '名詞', '記号', '記号', '名詞', '記号', '名詞', '記号', '助詞', '助詞', '名詞', '助詞', '動詞', '助詞', '記号', '名詞', '名詞', '名詞', '助詞', '名詞', '名詞', '助動詞', '名詞', '助詞', '助詞', '名詞', '名詞', '助詞', '動詞', '名詞', '助詞', '記号', '名詞', '名詞', '名詞', '助詞', '名詞', '名詞', '名詞', '名詞', '助詞', '名詞', '動詞', '名詞', '助詞', '動詞', '名詞', '助詞', '形容詞', '名詞', '名詞', '名詞', '名詞', '名詞', '助詞', '名詞', '助詞', '名詞', '名詞', '助詞', '名詞', '動詞', '助詞', '記号', '名詞', '名詞', '助詞', '名詞', '助詞', '副詞', '名詞', '名詞', '助動詞', '記号', '名詞', '助詞', '名詞', '動詞', '形容詞', '記号', '名詞', '助詞', '名詞', '動詞', '助詞', '名詞', '助詞', '動詞', '動詞', '記号', '名詞', '名詞', '助詞', '助詞', '名詞', '名詞', '記号', '名詞', '助詞', '助詞', '名詞', '名詞', '助詞', '動詞', '動詞', 

### 使用された単語の出現回数

In [6]:
cnt = collections.Counter(wakati)
for row in cnt:
    print(row+"\t= "+str(cnt[row]))

自然	= 12
言語	= 18
処理	= 7
（	= 5
し	= 4
ぜん	= 1
げん	= 1
ごしょ	= 1
り	= 1
、	= 21
英語	= 1
:	= 1
 	= 4
natural	= 1
language	= 1
processing	= 1
略称	= 1
：	= 1
NLP	= 1
）	= 5
は	= 12
人間	= 1
が	= 13
日常	= 1
的	= 7
に	= 10
使っ	= 1
て	= 10
いる	= 5
を	= 12
コンピュータ	= 3
さ	= 3
せる	= 2
一連	= 1
の	= 14
技術	= 1
で	= 3
あり	= 1
人工	= 1
知能	= 1
と	= 6
学	= 4
一	= 1
分野	= 1
ある	= 3
。	= 7
「	= 1
計算	= 2
」	= 1
computational	= 1
linguistics	= 1
類似	= 1
も	= 2
工学	= 1
な	= 5
視点	= 2
から	= 2
さす	= 2
に対して	= 1
重視	= 1
する	= 4
手法	= 2
事	= 1
多い	= 1
[	= 1
1	= 1
]。	= 1
データベース	= 1
内	= 1
情報	= 1
変換	= 4
たり	= 1
文章	= 1
より	= 1
形式	= 1
理解	= 7
やすい	= 1
表現	= 1
といった	= 1
含ま	= 2
れる	= 2
応用	= 1
例	= 1
として	= 1
予測	= 1
IME	= 1
など	= 5
文字	= 1
挙げ	= 1
られる	= 1

	= 1
こと	= 2
れ	= 2
差	= 1
意味	= 4
扱う	= 1
か	= 5
扱わ	= 1
ない	= 1
という	= 2
説	= 1
あっ	= 1
た	= 2
最近	= 1
数理	= 1
解析	= 7
統計	= 1
や	= 3
確率	= 1
広め	= 1
られ	= 1
為	= 1
パーサ	= 1
統語	= 1
器	= 1
一段と	= 1
精度	= 1
速度	= 1
上がり	= 1
その	= 1
意味合い	= 1
違っ	= 1
き	= 3
もともと	= 1
論	= 1
側面	= 1
全く	= 1
無視	= 1
達成	= 1
できる	= 1
非常	= 1
限ら	= 1
この	= 1
ため	= 1
形態素	= 1
構文	= 1
文脈	= 1
Syntax

### 使用された品詞の出現回数

In [7]:
cnt = collections.Counter(speech)
for row in cnt:
    print(row+"\t= "+str(cnt[row]))

名詞	= 156
記号	= 46
動詞	= 46
助動詞	= 15
助詞	= 99
形容詞	= 2
副詞	= 5
連体詞	= 3


## 構文解析
形態素解析の結果得られた単語に対して単語の関係性を解析する。

ただし、日本語に対応するライブラリは環境によって使用の可否が変わるため英語の文章で行う。

In [8]:
nltk.download("punkt")
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\decar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\decar\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [9]:
text = "There are two chickens in the garden."
words = nltk.word_tokenize(text)
words_parsed = nltk.pos_tag(words)
words_parsed

[('There', 'EX'),
 ('are', 'VBP'),
 ('two', 'CD'),
 ('chickens', 'NNS'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('garden', 'NN'),
 ('.', '.')]

得られた結果についてEXは存在、VBPは三人称単数以外の動詞、CDは基数、NNSは複数形の名刺、INは前置詞または従属接続詞、DTは限定詞、NNは名詞を表す。

なお、ここに現れていない他の構文解析結果の品詞はここを参照：https://www.ibm.com/docs/ja/wca/3.5.0?topic=analytics-part-speech-tag-sets

## TFとTF-IDFとトピック分析
自然言語を使った文書分類での特徴量では単語のTF値(出現頻度)やTF-IDF(出現頻度と文書逆頻度の積)を使用される。

ここではあらかじめ文章のジャンルは分からないものとして教師なし学習で単語のトピック分析を行う。

In [10]:
data = fetch_20newsgroups(remove=("headers", "footers", "quotes"))
n_topics = 20

### TF
文章における単語の出現頻度を特徴量とている。

In [11]:
tf_vec = CountVectorizer(max_features=1000, stop_words="english")
tf = tf_vec.fit_transform(data.data)

### TF-IDF
IDFは文章における逆頻度を指す。意図としては、単語の出現回数が多くてもどの文章にも表れている訳ではなく単語が他の文章に現れないレアな単語を探すことができる。

In [12]:
tfidf_vec = TfidfVectorizer(max_features=1000, stop_words="english")
tfidf = tfidf_vec.fit_transform(data.data)

### LDA

In [13]:
model_tf = LDA(n_components=n_topics)
model_tf.fit(tf)

LatentDirichletAllocation(n_components=20)

In [14]:
model_tfidf = LDA(n_components=n_topics)
model_tfidf.fit(tfidf)

LatentDirichletAllocation(n_components=20)

### TF値によるトピック分析

In [15]:
features = tf_vec.get_feature_names()
for i in range(20):
    print("topic %d:"%(i+1))
    row = model_tf.components_[i]
    for j in row.argsort()[:-20-1:-1]:
        print(features[j], end=" ")
    print()

topic 1:
people mr think government don right know going president just state rights stephanopoulos did make like want say israel time 
topic 2:
game just year good like team don think games time play got better season players know hockey ve really ll 
topic 3:
said people armenian didn know armenians just went don did time like told saw came took day say going started 
topic 4:
file program available image ftp files edu space pub nasa output entry information format directory info version source images use 
topic 5:
edu com 00 mail list send new email cs ca 50 sale offer shipping interested uk address ac article condition 
topic 6:
key chip encryption keys clipper government use security law public algorithm des escrow phone nsa secure bit number enforcement used 
topic 7:
data software bit mac use internet anonymous pc computer ibm hardware users available information mail os unix systems 32 standard 
topic 8:
scsi law jesus ide bus paul word david controller drive day mark john tran

### TF-IDFによるトピック分析

In [16]:
features = tfidf_vec.get_feature_names()
for i in range(20):
    print("topic %d:"%(i+1))
    row = model_tfidf.components_[i]
    for j in row.argsort()[:-20-1:-1]:
        print(features[j], end=" ")
    print()

topic 1:
ground wire women mr claim men news said posted saw explain la line certainly job details post did answer look 
topic 2:
drive card windows use video monitor disk mac like scsi pc memory software thanks apple problem does know used need 
topic 3:
key chip clipper encryption keys nsa phone algorithm escrow chips secure des government security secret use number privacy technology public 
topic 4:
car window cars use server like problem just windows engine good application know ve work don display road widget using 
topic 5:
00 10 11 15 12 20 23 25 14 17 april 30 13 18 50 21 16 19 1993 24 
topic 6:
com dave vs article game edu gov gm apr david ca 1993 93 ma al canada night 03 hockey internet 
topic 7:
mike hear 800 ll year pittsburgh just switch high better lines sure dc try make guess quality trade mode season 
topic 8:
mouse keyboard ac steve com thank uk weeks cable white newsgroup usenet thanks email ask page haven request mail simple 
topic 9:
government people gun armenian 

## 機械翻訳
分析対象あるいはモデルの作成について一つの言語で行っていても、他の言語でないと資料が無く和訳しないといけない事がある。そこで、GoogleTransAPIを使用して他言語で書かれた文章を対象の言語に翻訳する。ここでは英語のwikipediaの「Natural Langage Processing」について和訳する。

In [17]:
f = open("sample_en.txt", "r")
english = f.read()
english

'Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language, in particular how to program computers to process and analyze large amounts of natural language data. The goal is a computer capable of "understanding" the contents of documents, including the contextual nuances of the language within them. The technology can then accurately extract information and insights contained in the documents as well as categorize and organize the documents themselves.\n\nChallenges in natural language processing frequently involve speech recognition, natural-language understanding, and natural-language generation. '

In [18]:
trans = Translator()

In [19]:
trans.raise_Exception = True
japanese = trans.translate(english,dest="ja")
japanese.text

'自然言語処理（NLP）は、コンピューターと人間の言語間の相互作用に関係する言語学、コンピューターサイエンス、および人工知能のサブフィールド、特にコンピューターをプログラムして大量の自然言語データを処理および分析する方法です。目標は、文書の内容を「理解」できるコンピューターです。このテクノロジーは、ドキュメントに含まれる情報と洞察を正確に抽出し、ドキュメント自体を分類および整理することができます。\n\n自然言語処理の課題には、音声認識、自然言語の理解、自然言語の生成が頻繁に含まれます。'